In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2

# defining global variable path
image_path = "assets"

'''function to load folder into arrays and 
then it returns that same array'''
def load_images(path):
    list_of_files = []
    dirName = path
    for (dirpath, dirnames, filenames) in os.walk(dirName):
        list_of_files += [os.path.join(dirpath, file) for file in filenames]
    return list_of_files

In [3]:
from scipy import ndimage#
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

Using TensorFlow backend.


In [4]:
def process_image(image_path): 
    images = []
    list_image_paths = load_images(image_path)
    for path in list_image_paths: 
        image= (load_img(path))
        image.resize((640 ,480))
        images.append((img_to_array(image).flatten()))
        if i % 20 == 0: 
            print("image", i)
        #print(type(img))
        #print(img.mode)
    #img.show()
    return images

In [5]:
import gc
def load_images_from_list(image_list):
    images = []
    i = 1
    for image_path in image_list: 
        f = open(image_path, 'rb')
        image= (load_img(image_path))
        image = image.resize((640 ,480))
        image_arr = img_to_array(image).flatten()
        images.append(image_arr)
        image.close()
        del image
        gc.collect()
        if i % 20 == 0: 
            print("image", i)
        f.close()
        i += 1
        #print(type(img))
        #print(img.mode)
    #img.show()
    return images

In [6]:
import pandas as pd
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
df = pd.read_csv("fire_aug_database.csv")
df = shuffle(df)
df = df.drop(list(df.columns)[0], axis=1)
list_files=df["file"].tolist()
print(len(list_files))



314


In [7]:
X_images = np.array(load_images_from_list(list_files))
y = df["label"].to_list()
train_X, test_X, train_y, test_y = train_test_split(X_images, y, test_size=0.10, random_state=42)

image 20
image 40
image 60
image 80
image 100
image 120
image 140
image 160
image 180
image 200
image 220
image 240
image 260
image 280
image 300


In [8]:
parameters = {'bootstrap': True,
              'min_samples_leaf': 4,
              'n_estimators':30, 
              'min_samples_split': 10,
              'max_features': 'sqrt',
              'max_depth': 4,
              'max_leaf_nodes': None}

In [9]:
print("%d bytes" % (X_images.size * X_images.itemsize))

1157529600 bytes


In [10]:
RF_model = RandomForestClassifier(**parameters)

In [11]:
RF_model.fit(train_X, train_y)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=4, max_features='sqrt', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=4, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=30,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [13]:
from sklearn.metrics import accuracy_score
RF_predictions = RF_model.predict(test_X)
score = accuracy_score(test_y ,RF_predictions)
print(score)


0.84375


In [14]:
from sklearn.metrics import confusion_matrix
confusion_matrix(test_y, RF_predictions)

array([[18,  4],
       [ 1,  9]], dtype=int64)

In [15]:
from sklearn.metrics import accuracy_score
RF_predictions = RF_model.predict(train_X)
score = accuracy_score(train_y ,RF_predictions)
print(score)

0.9680851063829787


In [16]:
confusion_matrix(train_y, RF_predictions)

array([[137,   9],
       [  0, 136]], dtype=int64)

In [17]:
labels = df["label"]
y_fire = len(labels[labels == "fire"])
y_fire

168

In [18]:
y_no_fire = len(labels[labels == "no_fire"])
y_no_fire

146

In [19]:
distribution = y_fire/(y_no_fire+y_fire)
print(distribution)

0.535031847133758


# Algorithm in Action

## Fire images

In [20]:
import os
entries_fire = os.listdir('assets/addi/drone_shots')
entries_fire = ['assets/addi/fire_database/image31.jpg',
                "assets/addi/drone_shots/EC5F69EDF7433C9F50E8B4C445CD0806.JPG",
                'assets/addi/fire_database/image6.jpg',
                'assets/addi/fire_database/image88.jpg',
                'assets/addi/fire_database/image66.jpg',
                'assets/addi/fire_database/image656.jpg', 
                "assets/addi/fire_database/image666.jpg", 
                "assets/addi/fire_database/image799.jpg"]
X_images_prod = load_images_from_list(entries_fire)

In [21]:
print(entries_fire)
RF_predictions_prod = RF_model.predict(X_images_prod)
RF_predictions_prod

['assets/addi/fire_database/image31.jpg', 'assets/addi/drone_shots/EC5F69EDF7433C9F50E8B4C445CD0806.JPG', 'assets/addi/fire_database/image6.jpg', 'assets/addi/fire_database/image88.jpg', 'assets/addi/fire_database/image66.jpg', 'assets/addi/fire_database/image656.jpg', 'assets/addi/fire_database/image666.jpg', 'assets/addi/fire_database/image799.jpg']


array(['fire', 'fire', 'fire', 'fire', 'fire', 'fire', 'fire', 'fire'],
      dtype='<U7')

## No Fire 

In [22]:
import os
entries_fire = os.listdir('assets/addi/drone_shots')
entries_fire = ['assets/addi/fire_database/image168.jpg',
                
                'assets/addi/fire_database/image227.jpg',
                'assets/addi/fire_database/image235.jpg',
                'assets/addi/fire_database/image852.jpg', 
                "assets/addi/fire_database/image869.jpg", 
                "assets/addi/fire_database/image593.jpg"]
X_images_prod = load_images_from_list(entries_fire)

In [23]:
print(entries_fire)
RF_predictions_prod = RF_model.predict(X_images_prod)
RF_predictions_prod

['assets/addi/fire_database/image168.jpg', 'assets/addi/fire_database/image227.jpg', 'assets/addi/fire_database/image235.jpg', 'assets/addi/fire_database/image852.jpg', 'assets/addi/fire_database/image869.jpg', 'assets/addi/fire_database/image593.jpg']


array(['no_fire', 'no_fire', 'no_fire', 'no_fire', 'no_fire', 'no_fire'],
      dtype='<U7')

In [24]:
df = pd.read_csv("fire_data_large_database.csv")
list_files=df["file"].tolist()
X_images_prod = load_images_from_list(list_files)
label_test = df["label"].tolist()

image 20
image 40
image 60
image 80
image 100
image 120
image 140
image 160
image 180
image 200
image 220
image 240
image 260
image 280
image 300
image 320
image 340
image 360
image 380
image 400
image 420
image 440
image 460
image 480
image 500
image 520
image 540
image 560
image 580
image 600
image 620
image 640
image 660
image 680
image 700
image 720
image 740


In [25]:
RF_predictions = RF_model.predict(X_images_prod)
score = accuracy_score(label_test ,RF_predictions)
print(score)

0.47580645161290325


In [26]:
confusion_matrix(label_test, RF_predictions)

array([[151, 121],
       [269, 203]], dtype=int64)

# Fazit
+ Algorithm learns to detect smoke 
+ Algorithm does not necessarily detect fire 
+ Algorithm gets confused by fake smoke 
+ Enhance data base according to these findings 